## DL AFT Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/dl_pipeline')
from dl_pipeline import train_dl_complete

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_dl_'
model = 'aft'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'aft'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
model: aft
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6297765961668188
Integrated Brier Score: 0.1802507881066264
{'model': 'aft', 'dataset': 'METABRIC', 'cindex_train': [0.5947538583389341], 'cindex_test': [0.6297765961668188], 'ibs_train': [0.17538443016746047], 'ibs_test': [0.1802507881066264]}
1
model: aft
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6279769513658086
Integrated Brier Score: 0.17918164534835596
{'model': 'aft', 'dataset': 'METABRIC', 'cindex_train': [0.645788509942188], 'cindex_test': [0.6279769513658086], 'ibs_train': [0.16894859000344367], 'ibs_test': [0.17918164534835596]}
2
model: aft
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.562040322740519
Integrated Brier Score: 0.190692218183783
{'mo

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,aft,METABRIC,0.594754,0.629777,0.175384,0.180251
1,aft,METABRIC,0.645789,0.627977,0.168949,0.179182
2,aft,METABRIC,0.607591,0.562040,0.174568,0.190692
3,aft,METABRIC,0.632618,0.648640,0.171294,0.164039
4,aft,METABRIC,0.625705,0.621606,0.171495,0.180704
5,aft,FLCHAIN,0.783791,0.793876,0.099433,0.099221
6,aft,FLCHAIN,0.783310,0.794874,0.099926,0.098719
7,aft,FLCHAIN,0.794554,0.781971,0.096566,0.098769
8,aft,FLCHAIN,0.790914,0.767095,0.098580,0.102398
9,aft,FLCHAIN,0.785463,0.789665,0.100236,0.096366


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'aft'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
model: aft
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6093922651933702
Integrated Brier Score: 0.227233892779434
{'model': 'aft', 'dataset': 'BLCA', 'cindex_train': [0.7575869316715218], 'cindex_test': [0.6093922651933702], 'ibs_train': [0.20321940756409732], 'ibs_test': [0.227233892779434]}
1
model: aft
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6504642271982524
Integrated Brier Score: 0.20677894555232051
{'model': 'aft', 'dataset': 'BLCA', 'cindex_train': [0.7274717121127806], 'cindex_test': [0.6504642271982524], 'ibs_train': [0.19722600459267328], 'ibs_test': [0.20677894555232051]}
2
model: aft
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5555555555555556
Integrated Brier Score: 0.23636791654721995
{'model': 'aft

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)

In [9]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,aft,BLCA,0.757587,0.609392,0.203219,0.227234
1,aft,BLCA,0.727472,0.650464,0.197226,0.206779
2,aft,BLCA,0.659499,0.555556,0.207478,0.236368
3,aft,BLCA,0.713690,0.658648,0.203491,0.230672
4,aft,BLCA,0.772670,0.648998,0.211129,0.205170
5,aft,BRCA,0.674010,0.577691,0.209070,0.262107
6,aft,BRCA,0.739499,0.654301,0.179607,0.215542
7,aft,BRCA,0.773840,0.550373,0.169963,0.213288
8,aft,BRCA,0.843131,0.575113,0.171434,0.206162
9,aft,BRCA,0.809683,0.564283,0.153773,0.206813
